# 第9回講義 宿題

## 課題
RNN Encoder-Decoderにより高精度な英日翻訳器を実装してみましょう。

ネットワークの形などは特に制限を設けませんし、今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません。

## 目標値
BLEU：0.15

## ルール
- 以下のセルで指定されている`x_train, t_train`以外の学習データは使わないでください。
- `tokenizer_en, tokenizer_ja`は講義時と同様に`tensorflow.keras.preprocessing.text.Tokenizer`で英文と和文を指定したものになります。

## 提出方法
- 2つのファイルを提出していただきます。
  1. テストデータ (x_test) に対する予測ラベルを`submission_gen.csv`として保存し、Homeworkタブから**chap09**を選択して提出してください。
  2. それに対応するpythonのコードを`submission_code.py`として保存し、Homeworkタブから**chap09 (code)**を選択して提出してください。
    - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください。
    - writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください


- コードの内容を変更した場合は、1と2の両方を提出し直してください。

- なお、採点は1で行い、2はコードの確認用として利用します。(成績優秀者はコード内容を公開させていただくかもしれません)

- **宿題の締め切りは【出題週の翌週水曜日24時】です。**

# 注意 単語はid列で提出!!

## 評価方法

- 予測ラベルの（`t_testに対する`）BLEUスコア(4-gramまで)で評価します。
- 毎日24時にテストデータの一部に対するBLEUスコアでLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対するBLEUスコアでLeader Boardを更新します。これを最終的な評価とします。

## データの読み込み（このセルは修正しないでください）

In [1]:
import numpy as np
import pickle

def pickle_load(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

def load_data():
    # 学習データ
    x_train = pickle_load('/root/userspace/public/chap09/data/x_train.pkl')
    t_train = pickle_load('/root/userspace/public/chap09/data/t_train.pkl')
    tokenizer_en = np.load('/root/userspace/public/chap09/data/tokenizer_en.npy').item()
    tokenizer_ja = np.load('/root/userspace/public/chap09/data/tokenizer_ja.npy').item()
 
    # テストデータ
    x_test = pickle_load('/root/userspace/public/chap09/data/x_test.pkl')

    return (x_train, t_train, tokenizer_en, tokenizer_ja, x_test)

x_train, t_train, tokenizer_en, tokenizer_ja, x_test = load_data()

## 実装

In [ ]:
# %%writefile /root/userspace/chap09/submission/submission_code.py
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import csv

### レイヤー定義 ###
class Embedding:
    # WRITE ME
    def __init__(self, emb_dim, )

# WRITE ME

### グラフ構築 ###
tf.reset_default_graph()

emb_dim = 1
hid_dim = 1
pad_index = 0

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.int32, [None, None], name='t')

# WRITE ME

### データの準備 ###
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train)

### 学習 ###
n_epochs = 1
batch_size = 1

# WRITE ME

### 生成用グラフ構築 ###
max_len = tf.placeholder(tf.int32, name='max_len') # iterationの繰り返し回数の限度

def cond():
    # WRITE ME

def body():
    # WRITE ME

# WRITE ME

### 生成 ###
bos_id_ja, eos_id_ja = tokenizer_ja.texts_to_sequences(['<s> </s>'])[0]
y_pred = sess.run(
    # WRITE ME
)

### 出力 ###
def get_raw_contents(dataset, num, bos_id, eos_id):
    result = []
    for index in dataset[num]:
        if index == eos_id:
            break
            
        result.append(index)
        
        if index == bos_id:
            result = []
            
    return result

output = [get_raw_contents(y_pred, i, bos_id_ja, eos_id_ja) for i in range(len(y_pred))]

with open('/root/userspace/chap09/materials/submission_gen.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(output)

In [ ]:
##### 課題を終わらせる用 ######
# %%writefile /root/userspace/chap09/submission/submission_code.py
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import csv


### 関数定義 ###
bos_id_ja, eos_id_ja = tokenizer_ja.texts_to_sequences(['<s> </s>'])[0]

def get_raw_contents(dataset, num, bos_id, eos_id):
    result = []
    for index in dataset[num]:
        if index == eos_id:
            break
            
        result.append(index)
        
        if index == bos_id:
            result = []
            
    return result
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))


### レイヤー定義 ###
class Embedding:
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        self.V = tf.Variable(tf.random_normal([vocab_size, emb_dim], stddev=scale), name='V')

    def __call__(self, x):
        return tf.nn.embedding_lookup(self.V, x) # nn.embedding_lookupが授業で扱った「速くなるポイント」

class LSTM:
    # hid_dim: 隠れ層の次元数
    # seq_len: 系列長を示すベクトル size=[batch_size]
    # initial_state: 隠れ層の初期値 通常はHe Initializationとかで決める感じ
    # return_state: 最終期のstateを返すかどうか
    # return_sequences: 全ての期における隠れ層の数値を格納したテンソルを返すかどうか size = [batch_size, enc_length, hid_dim]
    # hold_state: 順伝播後のdecoder_lstmなどのstateを取り出したいときにTrueにする。
    def __init__(self, hid_dim, seq_len, initial_state, return_state = False, return_sequences = False, hold_state = False, name = None):
        self.cell = tf.nn.rnn_cell.BasicLSTMCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len
        self.return_state = return_state
        self.return_sequences = return_sequences
        self.hold_state = hold_state
        self.name = name

    def __call__(self, x):
        with tf.variable_scope(self.name):
            # 各時刻の全ての出力を抽出している
            outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        
        if self.hold_state:
            self.initial_state = state
        
        if not self.return_sequences:
            outputs = state.h
            
        if not self.return_state:
            return outputs
        
        return outputs, state

class Attention:
    def __init__(self, hid_dim, out_dim, enc_out, seq_len):
        e_hid_dim, d_hid_dim = hid_dim, hid_dim
        
        self.enc_out = enc_out 
        self.seq_len = seq_len 
        glorot_a = tf.cast(tf.sqrt(6/(e_hid_dim + d_hid_dim)), tf.float32) # 初期化の値を計算
        self.W_a  = tf.Variable(tf.random_uniform([e_hid_dim, d_hid_dim], minval=-glorot_a, maxval=glorot_a), name='W_a') # 重み行列
        
        glorot_c = tf.cast(tf.sqrt(6/(e_hid_dim + d_hid_dim + out_dim)), tf.float32)
        self.W_c = tf.Variable(tf.random_uniform([e_hid_dim+d_hid_dim, out_dim], minval=-glorot_c, maxval=glorot_c), name='W_c')
        self.b    = tf.Variable(np.zeros([out_dim]).astype('float32'), name='b')
        
    def __call__(self, dec_out): # 順伝播を記述する部分
        # Attention score h_t^T @ W_a @ \bar{h}_s
        # self.enc_out: [batch_size, enc_length, e_hid_dim]
        # self.W_a  : [e_hid_dim, d_hid_dim]
        # -> enc_out: [batch_size, enc_length, d_hid_dim]
        enc_out = tf.einsum("ijk, km -> ijm", self.enc_out, self.W_a) # einsumを使わない書き方はanswer参照
        
        # W_a_broadcasted = tf.tile(tf.expand_dims(self.W_a, axis=0), [tf.shape(self.enc_out)[0],1,1])
        # enc_out = tf.matmul(self.enc_out, W_a_broadcasted)
        
        # dec_out: [batch_size, dec_length, d_hid_dim]
        # enc_out: [batch_size, enc_length, d_hid_dim]
        # -> score: [batch_size, dec_length, enc_length]
        score = tf.einsum("nid, njd -> nij", dec_out, enc_out)
        
        # WRITE ME
        
        # Attention weight
        # 正規化を行う必要性がある
        score = score - tf.reduce_max(score, axis = -1, keep_dims = True) # For stable softmax computation
        # self.seq_len: [batch_size] で第i成分がi番目の文の系列長になっている
        # mask: [batch_size, dec_length, enc_length]になっていることに注意
        mask = tf.cast(tf.sequence_mask(self.seq_len, tf.shape(score)[-1]), tf.float32) # encoder mask
        exp_score = tf.exp(score) * tf.expand_dims(mask, axis = 1)
        self.a = exp_score / tf.reduce_sum(exp_score, axis = 2, keep_dims = True) # softmax!
        
        # WRITE ME
        
        # Context vector
        # Recall that...
        # self.enc_out: [batch_size, enc_length, e_hid_dim]
        # self.a: [batch_size, dec_length, enc_length]
        # -> c: [batch_size, dec_length, e_hid_dim]
        c = tf.einsum("ijk, imj -> imk", self.enc_out, self.a)
        
        # self.W_c: [e_hid_dim+d_hid_dim, out_dim]
        # expand_dims: -> [1, e_hid_dim+d_hid_dim, out_dim]
        # tile(,[batch_size, 1, 1]): -> [batch_size, e_hid_dim+d_hid_dim, out_dim]
        W_c_broadcasted = tf.tile(tf.expand_dims(self.W_c, axis=0), [tf.shape(c)[0],1,1])
        
        # c, dec_out: [batch_size, dec_length, e_hid_dim], [batch_size, dec_length, d_hid_dim]
        # tf.concat([c, dec_out], -1): -> [batch_size, dec_length, e_hid_dim + d_hid_dim]
        return tf.nn.tanh(tf.matmul(tf.concat([c, dec_out], -1), W_c_broadcasted) + self.b)
    
# WRITE ME

### グラフ構築 ###
tf.reset_default_graph()

emb_dim = 512
hid_dim = 256
att_dim = 128
pad_index = 0

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.int32, [None, None], name='t')
seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)
seq_len_t_in = tf.reduce_sum(tf.cast(tf.not_equal(t, pad_index), tf.int32), axis=1) - 1 # 終端記号は含めないので-1する

t_out = tf.one_hot(t[:, 1:], depth=ja_vocab_size, dtype=tf.float32) # ラベルデータの文章部分をone_hot表現に直す。
t_out = t_out * tf.expand_dims(tf.cast(tf.not_equal(t[:, 1:], pad_index), tf.float32), axis=-1) # padding部分はmaskする

initial_state = tf.nn.rnn_cell.LSTMStateTuple(tf.zeros([tf.shape(x)[0], hid_dim]), tf.zeros([tf.shape(x)[0], hid_dim]))

### Encoder ###
h_e = Embedding(en_vocab_size, emb_dim)(x) # Embedded inputを作成(入力の完成)
encoded_outputs, encoded_state = LSTM(hid_dim, seq_len, initial_state, return_sequences = True, return_state=True, name='encoder_lstm')(h_e) # LSTMに通す

### Decoder ###
decoder = [
    # WRITE ME
    Embedding(ja_vocab_size, emb_dim),
    LSTM(hid_dim, seq_len_t_in, encoded_state, return_sequences = True, name = 'decoder_lstm_a'),
    Attention(hid_dim, att_dim, encoded_outputs, seq_len),
    tf.layers.Dense(ja_vocab_size, tf.nn.softmax)
]

h_d = decoder[0](t)
h_d = decoder[1](h_d)
h_d = decoder[2](h_d)
y = decoder[3](h_d)

cost = -tf.reduce_mean(tf.reduce_sum(t_out * tf_log(y[:, :-1]), axis=[1, 2]))
train = tf.train.AdamOptimizer().minimize(cost)

### データの準備 ###
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train)

### 学習 ###
n_epochs = 15
batch_size = 125
n_batches_train = math.ceil(len(x_train)/batch_size)
n_batches_valid = math.ceil(len(x_valid)/batch_size)
# num_batches_valid = x_valid//batch_size ?

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(n_epochs):
    # Train
    train_costs = []
    for i in range(n_batches_train):
        start = i * batch_size
        end = min(start + batch_size, len(x_train))
        
        x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index))
        t_train_batch = np.array(pad_sequences(t_train[start:end], padding='post', value=pad_index))
        
        _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
        train_costs.append(train_cost)

    # Valid
    valid_costs = []
    for i in range(n_batches_valid):
        start = i * batch_size
        end = min(start + batch_size, len(x_valid))
        
        x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index))
        t_valid_pad = np.array(pad_sequences(t_valid[start:end], padding='post', value=pad_index))
        
        valid_cost = sess.run(cost, feed_dict={x: x_valid_pad, t: t_valid_pad})
        valid_costs.append(valid_cost)

    print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs)))
# WRITE ME

### 生成用グラフ構築 ###
bos_eos = tf.placeholder(tf.int32, [2], name='bos_eos')
max_len = tf.placeholder(tf.int32, name='max_len') # iterationの繰り返し回数の限度

def cond():
    unfinished = tf.not_equal(tf.reduce_sum(tf.cast(continue_frag, )))
    # WRITE ME

def body():
    # WRITE ME

# WRITE ME

### 生成 ###
bos_id_ja, eos_id_ja = tokenizer_ja.texts_to_sequences(['<s> </s>'])[0]
y_pred = sess.run(
    # WRITE ME
)

### 出力 ###
def get_raw_contents(dataset, num, bos_id, eos_id):
    result = []
    for index in dataset[num]:
        if index == eos_id:
            break
            
        result.append(index)
        
        if index == bos_id:
            result = []
            
    return result

output = [get_raw_contents(y_pred, i, bos_id_ja, eos_id_ja) for i in range(len(y_pred))]

with open('/root/userspace/chap09/materials/submission_gen.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(output)